# Initiate spark

In [3]:
import os

from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, max, to_date, expr

from delta.tables import DeltaTable

conf = (
    SparkConf()
    .setAppName("Spark minIO Test")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))
    .set("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
spark = SparkSession(sc).builder.getOrCreate()

22/10/16 09:01:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
from delta.tables import DeltaTable

In [5]:
df = spark.read.format('delta').load('s3a://silver-knmi/uurgegevens')

22/10/16 09:01:44 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
df.show()

22/10/16 09:01:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+-----+------------------------------+-----------------------------------+----------------------------------------------------+---------------------------------+-----------+-----------------------------+---------------------+-----------------+-----------------------------+-------------------------------+-------------------------+------------------------+---------------+----------------------+--------+------------+------------------------+-------------+--------------+--------------+-----------------+-------------+--------------------+
|weather_station_code|      date| hour|mean_wind_direction_in_degrees|mean_windspeed_in_meters_per_second|mean_windspeed_in_meters_per_second_last_ten_minutes|max_windgust_in_meters_per_second|temperature|min_temp_at_10cm_above_ground|dew_point_temperature|sunshine_duration|global_radiation_in_J_per_cm2|precipitation_duration_in_hours|precipitaion_amount_in_mm|sealevel_pressure_in_hPa|visibility_in_m|cloud_cover_in_octants|h

In [7]:
df.coalesce(1).write.format("delta").save('s3a://test-bucket/uurgegevens')

In [8]:
df_stations = spark.read.format('delta').load('s3a://silver-knmi/weather_stations')

In [15]:
df_stations.coalesce(1).write.format("delta").mode('overwrite').save('s3a://test-bucket/weather_stations')

In [16]:
df_date = spark.read.format('delta').load('s3a://silver-generated/dim_date')

In [17]:
df_date.coalesce(1).write.format("delta").save('s3a://test-bucket/dim_date')